__Name:__ Tonson Praphabkul  
__Student id:__ st123010

### Lab report
&emsp; In this lab, I experiments with 4 models, which is "AlexNet with LRN", "Alexnet without LRN", "GoogleNet" and "Pre-trained GoogleNet" from torchvision. The dataset that I used to experiments in this lab is CIFAR-10. I has transformed all of the input data with the code below.  
  
preprocess = transforms.Compose([  
&emsp;transforms.Resize(256),  
&emsp;transforms.CenterCrop(224),  
&emsp;transforms.ToTensor(),  
&emsp;transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])  

&emsp; The input data got seperated into train set (40,000), validation set (10,000), test set (10,000) roughly 66%, 16%, 16% respectively. The batch size for each dataloaders is 4 and only shuffle the train set. The criterion for the model is CrossEntropyLoss and optimizer is Stochastic gradient descent with learning rate of 0.0001 and momentum of 0.9. I trained those model for 10 epochs each. The results is display at the table below.


|         Model        | Train Accuracy | Validation accruacy | Test accruacy | Number of parameters | Time used |
|:--------------------:|----------------|:-------------------:|:-------------:|----------------------|-----------|
| AlexNet with LRN     | 0.61           | 0.63                | 0.63          | 58,322,314           | 30m 44s   |
| Alexnet without LRN  | 0.72           | 0.73                | 0.73          | 58,322,314           | 31m 25s   |
| GoogleNet            | 0.83           | 0.82                | 0.82          | 10,635,134           | 114m 10s  |
| Pretrained_GoogleNet | 0.96           | 0.94                | 0.93          | 13,004,888           | 91m 25s   |


&emsp; There are few interesting things that I liked to point out. First, AlexNet with LRN is performs worse than AlexNet without LRN. My assumption is in this case some important information got loss when LRN adjusts/contrast the brightness. Second, GoogleNet takes way more time to train than AlexNet even with far less number of parameters. I think it come from the complexity of neural network in term of height. GoogleNet models is far more deeper than Alexnet.
  
&emsp; The pretrain GoogleNet from torchvision performs the best, far surpass other model. The number of parameters of pretrained GoogleNet is higher than the GoogleNet that I usded (modify from the lab provided code). All of the models seem to have no problems with generalization and I think if I increases the number of epoch, we can still increase the accruacy. 


In [ ]:
class Alexnet_Wo_LRN(nn.Module):
    
    def __init__(self, output_size):
        super().__init__()  #inherit everything from nn.Module
        self.layer1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.layer2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.layer3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.layer5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.layer6 = nn.Linear(256 * 6 * 6, 4096)
        self.layer7 = nn.Linear(4096, 4096)
        self.layer8 = nn.Linear(4096, output_size)
        self.Adapavgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.relu   = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout = nn.Dropout()

    def flatten(self, input_flatten):
        batch_size = input_flatten.shape[0]
        return input_flatten.view(batch_size, -1)

    def forward(self, x):

        out = self.layer1(x)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer2(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer3(out)
        out = self.relu(out)

        out = self.layer4(out)
        out = self.relu(out)

        out = self.layer5(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.Adapavgpool(out)
        out = self.flatten(out)
        out = self.dropout(out)

        out = self.layer6(out)
        out = self.relu(out)

        out = self.dropout(out)
        out = self.layer7(out)
        out = self.relu(out)

        out = self.layer8(out)

        return out

In [ ]:
class Alexnet_LRN(nn.Module):
    
    def __init__(self, output_size):
        super().__init__()  #inherit everything from nn.Module
        self.layer1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.layer2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.layer3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.layer5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.layer6 = nn.Linear(256 * 6 * 6, 4096)
        self.layer7 = nn.Linear(4096, 4096)
        self.layer8 = nn.Linear(4096, output_size)
        self.Adapavgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.relu   = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout = nn.Dropout()
        self.lrn     = nn.LocalResponseNorm(5, alpha=0.00001, beta = 0.75, k = 2.0)

    def flatten(self, input_flatten):
        batch_size = input_flatten.shape[0]
        return input_flatten.view(batch_size, -1)

    def forward(self, x):

        out = self.layer1(x)
        out = self.relu(out)
        out = self.lrn(out)
        out = self.maxpool(out)

        out = self.layer2(out)
        out = self.relu(out)
        out = self.lrn(out)
        out = self.maxpool(out)

        out = self.layer3(out)
        out = self.relu(out)

        out = self.layer4(out)
        out = self.relu(out)

        out = self.layer5(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.Adapavgpool(out)
        out = self.flatten(out)
        out = self.dropout(out)

        out = self.layer6(out)
        out = self.relu(out)

        out = self.dropout(out)
        out = self.layer7(out)
        out = self.relu(out)

        out = self.layer8(out)

        return out

In [ ]:
class Inception(nn.Module):
    '''
    Inception block for a GoogLeNet-like CNN

    Attributes
    ----------
    in_planes : int
        Number of input feature maps
    n1x1 : int
        Number of direct 1x1 convolutions
    n3x3red : int
        Number of 1x1 reductions before the 3x3 convolutions
    n3x3 : int
        Number of 3x3 convolutions
    n5x5red : int
        Number of 1x1 reductions before the 5x5 convolutions
    n5x5 : int
        Number of 5x5 convolutions
    pool_planes : int
        Number of 1x1 convolutions after 3x3 max pooling
    b1 : Sequential
        First branch (direct 1x1 convolutions)
    b2 : Sequential
        Second branch (reduction then 3x3 convolutions)
    b3 : Sequential
        Third branch (reduction then 5x5 convolutions)
    b4 : Sequential
        Fourth branch (max pooling then reduction)
    '''
    
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        self.in_planes = in_planes
        self.n1x1 = n1x1
        self.n3x3red = n3x3red
        self.n3x3 = n3x3
        self.n5x5red = n5x5red
        self.n5x5 = n5x5
        self.pool_planes = pool_planes
        
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
            

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1, y2, y3, y4], 1) # <<<< dim 1 = channel

class Aux_layer(nn.Module):
    def __init__(self, position):    
        super(Aux_layer, self).__init__()
        
        self.avgpool = nn.AvgPool2d(5, stride=3) 
        self.position = position
        
        if self.position == 'a4':
            self.conv = nn.Sequential(nn.Conv2d(512,128,kernel_size = 1,stride = 1), nn.ReLU(True))   
        else:
            self.conv = nn.Sequential(nn.Conv2d(528,128,kernel_size = 1,stride = 1), nn.ReLU(True))   

        self.fc1 = nn.Sequential(nn.Linear(2048, 1024), nn.ReLU(True), nn.Dropout(0.7))
        self.fc2 = nn.Linear(1024, 10)
    
    def forward(self,x):
        aux = self.avgpool(x)
        aux = self.conv(aux)
#         print(f'aux conv: {aux.shape}')
        aux = aux.flatten(start_dim = 1)
#         print(f'aux: {aux.shape}')
        aux = self.fc1(aux)
        aux = self.fc2(aux)
        return aux

class GoogLeNet(nn.Module):
    '''
    GoogLeNet-like CNN

    Attributes
    ----------
    pre_layers : Sequential
        Initial convolutional layer
    a3 : Inception
        First inception block
    b3 : Inception
        Second inception block
    maxpool : MaxPool2d
        Pooling layer after second inception block
    a4 : Inception
        Third inception block
    b4 : Inception
        Fourth inception block
    c4 : Inception
        Fifth inception block
    d4 : Inception
        Sixth inception block
    e4 : Inception
        Seventh inception block
    a5 : Inception
        Eighth inception block
    b5 : Inception
        Ninth inception block
    avgpool : AvgPool2d
        Average pool layer after final inception block
    linear : Linear
        Fully connected layer
    '''

    def __init__(self):
        super(GoogLeNet, self).__init__()
        
        self.is_debug = False
        
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3,64, kernel_size=7, stride =2, padding =3),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride =2, padding = 1),
            nn.LocalResponseNorm(5, alpha = 0.0001, beta = 0.75, k = 2.0),
            nn.Conv2d(64, 64, kernel_size=1),
            nn.ReLU(True),
            nn.Conv2d(64, 192, kernel_size=3, stride = 1, padding=1),
            nn.ReLU(True),
            nn.LocalResponseNorm(5, alpha = 0.0001, beta = 0.75, k = 2.0),              
            nn.MaxPool2d(3, stride =2, padding =1),
        )

        self.a3 = Inception(192,  64,  96, 128, 16, 32, 32)
        self.b3 = Inception(256, 128, 128, 192, 32, 96, 64)

        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = Inception(480, 192,  96, 208, 16,  48,  64)
        self.b4 = Inception(512, 160, 112, 224, 24,  64,  64)
        self.c4 = Inception(512, 128, 128, 256, 24,  64,  64)
        self.d4 = Inception(512, 112, 144, 288, 32,  64,  64)
        self.e4 = Inception(528, 256, 160, 320, 32, 128, 128)

        self.a5 = Inception(832, 256, 160, 320, 32, 128, 128)
        self.b5 = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool= nn.AvgPool2d(7, stride=1) ## orig = 8
        self.dropout = nn.Dropout(0.4)
        self.linear = nn.Linear(1024, 10)
        
        self.aux_a4 = Aux_layer('a4')
        self.aux_d4 = Aux_layer('d4')
    
    def forward(self, x):
        out = self.pre_layers(x)
        
        if self.is_debug : print(f'pre-layer: {out.shape}')
            
        out = self.a3(out)
        if self.is_debug : print(f'a3: {out.shape}')
            
        out = self.b3(out)
        if self.is_debug : print(f'b3 :{out.shape}')
            
        out = self.maxpool(out)
        if self.is_debug : print(f'maxpool: {out.shape}')
            
        out = self.a4(out)
        if self.is_debug : print(f'a4: {out.shape}')
            
        aux_a4 = self.aux_a4(out)
        out = self.b4(out)
        if self.is_debug : print(f'b4 : {out.shape}')
            
        out = self.c4(out)
        if self.is_debug : print(f'c4 : {out.shape}') 
            
        out = self.d4(out)
        if self.is_debug : print(f'd4 : {out.shape}')
            
        aux_d4 = self.aux_d4(out)
        out = self.e4(out)
        if self.is_debug : print(f'e4 : {out.shape}')
            
        out = self.maxpool(out)
        if self.is_debug : print(f'maxpool : {out.shape}')
            
        out = self.a5(out)
        if self.is_debug : print(f'a5 : {out.shape}')
            
        out = self.b5(out)
        if self.is_debug : print(f'b5 : {out.shape}')
            
        out = self.avgpool(out)
        if self.is_debug : print(f'avgpool : {out.shape}')
            
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        if self.training == True:
            return out, aux_a4, aux_d4
        else:
            return out